# Data Analysis with FugueSQL on Coiled Dask Clusters

In this notebook we will discuss [fugue-sql](https://docs.dask.org/en/latest/dataframe-sql.html#does-dask-implement-sql), an abstraction layer that allows users to run SQL queries on top of Pandas, Spark, and Dask dataframes. fugue-sql is part of the broader [fugue project](https://github.com/fugue-project/fugue), which aims to be an abstaction layer for distributed compute workflows. Fugue has both a Python and SQL interface. Users can choose the engine to run on just by specifying.

FugueSQL is meant for data analysts and SQL lovers

<img src="https://raw.githubusercontent.com/fugue-project/fugue/master/images/logo.svg" align="left" width="250"/>

In [1]:
import pandas as pd
import dask.dataframe as dd

In [2]:
import coiled

cluster = coiled.Cluster(
    n_workers=10,
    software="kvnkho/fugue-sql",
)
cluster

# cluster = coiled.Cluster(n_workers=10)
# cluster

Output()

Checking environment images
Valid environment image found


In [3]:
from dask.distributed import Client

client = Client(cluster)
client

/opt/miniconda3/envs/fugue-sql/lib/python3.7/site-packages/distributed/client.py:1140: VersionMismatchWarning: Mismatched versions found

+-------------+-----------+-----------+-----------+
| Package     | client    | scheduler | workers   |
+-------------+-----------+-----------+-----------+
| distributed | 2021.04.0 | 2021.03.1 | 2021.03.1 |
| numpy       | 1.19.2    | 1.20.2    | 1.20.2    |
+-------------+-----------+-----------+-----------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Client Scheduler: tls://ec2-3-15-211-21.us-east-2.compute.amazonaws.com:8786 Dashboard: http://ec2-3-15-211-21.us-east-2.compute.amazonaws.com:8787,Cluster Workers: 9 Cores: 36 Memory: 154.62 GB


## Setup

fugue-sql can be imported in notebooks by using the `fugue_notebook.setup` function. This provides syntax highlighting for fugue-sql cells and allows us to use the `%%fsql` magic.

At the moment, the notebook extension is only available for traditional iPython notebooks. This means syntax highlighting will fail in JupyterLab environments.

In [4]:
from fugue_notebook import setup
try:
    setup()
except:
    print("Syntax highlighting not yet available for JupyterLab")

## Initial Look



Here we load in the NYC taxi data with dask. This is standard from other demos. Persist prevents re-reading unnecessarily.

In [5]:
taxi_df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv",
    dtype={'RatecodeID': 'float64',
           'VendorID': 'float64',
           'passenger_count': 'float64',
           'payment_type': 'float64'},
    storage_options={"anon": True},
    blocksize="16 MiB",
).persist()

Here is our first look at fugue-sql. Notice the `%%fsql` cell magic turns the whole cell into a SQL cell. `%%fsql dask` tells Fugue to use Dask as an execution engine. Not specifying an execution engine defaults to pandas.

The Dask DataFrame (named `df`) that was previously loaded in is now accessible by this SQL cell. `fugue-sql` has all ANSI SQL keywords available, so here we show a simple `GROUP BY` and `ORDER BY`. We also use the AVG aggregate function.

For iPython notebook users, syntax highlighting for SQL keywords will be available.

In [6]:
%%fsql dask
-- This is a SQL cell
tempdf = SELECT passenger_count, AVG(tip_amount) AS average_tip
           FROM taxi_df
       GROUP BY passenger_count

  SELECT *
    FROM tempdf
ORDER BY passenger_count ASC
   LIMIT 5
   PRINT

,passenger_count,average_tip
0,0.0,1.786901
1,1.0,1.828308
2,2.0,1.833877
3,3.0,1.795579
4,4.0,1.702710


## Basics of fugue-sql

fugue-sql aims to make SQL easier to use. Some enhancements have been added. One of them is already visible above where we assigned a query to `tempdf` .

For familiar Dask users, this is a delayed execution, as Dask constructs a DAG.

### Passing Python DataFrames to fugue-sql

`fugue-sql` supports Python interoperatibility. DataFrames defined outside `%%fsql` cells can be used inside the SQL queries. In this example, we create an example DataFrame and use it inside a following `fugue-sql` code block.

In [7]:
example = pd.DataFrame({'a':[1,2,3],'b':[1,2,3]})

### Passing fugue-sql results to Python

By default, the DataFrames inside fugue-sql cells will not be accessible by Python cells (or even by succeeding fugue-sql cells). We have to use the `YIELD DATAFRAME` keyword to make a DataFrame available in memory. For significantly large DataFrames, we can use the `YIELD FILE` keyword. This saves the file in a temporary location and loads it when needed.

In [8]:
%%fsql
SELECT * 
  FROM example
 WHERE a > 1
 YIELD DATAFRAME AS filtered_example

In [9]:
%%fsql
SELECT *
  FROM filtered_example
 PRINT

,a,b
0,2,2
1,3,3


In [10]:
# Using filtered_example from a fugue-sql cell above
filtered_example.as_pandas().head()

,a,b
1,2,2
2,3,3


### Load and Save

fugue-sql allows users to `LOAD` from csv/json/parquet files using Pandas and Dask under the hood. This means we can load in data, perform transformations on it, and then `SAVE` the results. This allows data analysts to work on flat files with a language they are comfortable in.

In [12]:
%%fsql
SELECT * FROM example
SAVE OVERWRITE "/tmp/f.csv" (header=TRUE)

loaded_example = LOAD "/tmp/f.csv" (header=TRUE)
PRINT 

,a,b
0,1,1
1,2,2
2,3,3


### Jinja Templating

In [13]:
n = 1

In [14]:
%%fsql
SELECT *
  FROM example
 WHERE a = {{n}}
 PRINT

,a,b
0,1,1


### Altering Schema

Note that if we don't infer the schema, Pandas loads most columns as strings. We can use `ALTER COLUMNS` to change the schema. For DataFrames with a large number of columns, we recommend using `infer_schema` and then `ALTER COLUMNS` to ensure the correct types.

In [20]:
%%fsql
df = LOAD "/tmp/f.csv" (header = TRUE, infer_schema=TRUE) COLUMNS a:str, b:str
df = ALTER COLUMNS a:int, b:str FROM df

### Anonymity and Inline

Anonymity is when the DataFrame to perform the operation on is not specified. As a default, the output of the last operation will be used. This is a `fugue-sql` feature designed to simplify code. PRINT is an example of this.

Inline statements are wrapping another SQL statement inside parenthesis so that they are evaluated first.

In [21]:
%%fsql
df = SELECT * FROM (LOAD "/tmp/f.csv" (header=TRUE))
ALTER COLUMNS a:int, b:double
PRINT 5 ROWS

,a,b
0,1,1.0
1,2,2.0
2,3,3.0


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


In [ ]:
### PREPARTITION and PRESORT

In [ ]:
### fsql function

## Sample Data Analysis

Now that we have covered the basics, we will perform a simple analysis and show some more advance features such as integration Python code and utilizing the Dask execution engine more. First we start by displaying the head of our DataFrame

In [ ]:
%%fsql

SELECT *
  FROM taxi_df
 PRINT 10 ROWS

## More information

Fugue is a pure abstraction layer that makes code portable across differing computing frameworks such as Pandas, Spark and Dask. It decouples logic from execution engines, allowing users to write code without worrying about the execution engine during runtime. All questions are welcome in the Slack channel.

[Fugue Repo](https://github.com/fugue-project/fugue)

[Fugue Slack](https://join.slack.com/t/fugue-project/shared_invite/zt-jl0pcahu-KdlSOgi~fP50TZWmNxdWYQ)